In [1]:
from keras.models import Sequential, model_from_json
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.constraints import maxnorm
from keras import regularizers
from keras.layers.normalization import BatchNormalization

from keras.layers import *
from keras.models import *

from keras.regularizers import l2

from keras.applications import imagenet_utils, Xception
from keras.preprocessing.image import img_to_array
import keras.backend as K

import numpy as np 
import json
import os
import cv2
import h5py
import pickle

import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [2]:
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)

In [3]:
from helpers import TrainingMonitor
from helpers import Utils

In [4]:
Xoh_file = open('../input/dataset/Xoh.pickle', 'rb')
x = pickle.load(Xoh_file)
Xoh_file.close()

In [5]:
Yoh_file = open('../input/dataset/Yoh.pickle', 'rb')
y = pickle.load(Yoh_file)
Yoh_file.close()

In [6]:
x[0].shape

(30, 37)

In [7]:
y[0].shape

(10, 11)

In [8]:
train_split, val_split = 0.7, 0.15

train_len = int(train_split * x.shape[0])
train_val_len = int((train_split + val_split) * x.shape[0])

x_train = x[:train_len]
x_val = x[train_len:train_val_len]
x_test = x[train_val_len:]

y_train = y[:train_len]
y_val = y[train_len:train_val_len]
y_test = y[train_val_len:]

In [9]:
x_train.shape

(700, 30, 37)

In [10]:
TIME_STEPS = x_train.shape[1]
INPUT_DIM = x_train.shape[2]

In [11]:
y_train.shape

(700, 10, 11)

In [12]:
def attention(inputs):
    input_dim = int(inputs.shape[2])
    a = Permute((2, 1))(inputs)
    a = Dense(TIME_STEPS, activation='softmax')(a)
    a_probs = Permute((2, 1))(a)
    output_attention_mul = multiply([inputs, a_probs])
    return output_attention_mul

In [15]:
inputs = Input(shape=(TIME_STEPS, INPUT_DIM,))
attention_mul = attention(inputs)
output = LSTM(128)(attention_mul)
output = RepeatVector(10)(output)
output = LSTM(64, return_sequences=True)(output)
output = TimeDistributed(Dense(11, activation="softmax"))(output)
model = Model(inputs, output)

In [16]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 30, 37)       0                                            
__________________________________________________________________________________________________
permute_3 (Permute)             (None, 37, 30)       0           input_2[0][0]                    
__________________________________________________________________________________________________
dense_3 (Dense)                 (None, 37, 30)       930         permute_3[0][0]                  
__________________________________________________________________________________________________
permute_4 (Permute)             (None, 30, 37)       0           dense_3[0][0]                    
__________________________________________________________________________________________________
multiply_2

In [17]:
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"]) 

In [18]:
output_path = "../output/"

print(os.getpid())
filepath=output_path + "progress/seq2seq-attention-weights-best.hdf5"
MC = ModelCheckpoint(filepath, monitor='val_acc', verbose=0, save_best_only=True, mode='max')

figPath = os.path.sep.join([output_path, "monitor/{}.png".format(os.getpid())])
jsonPath = os.path.sep.join([output_path, "monitor/{}.json".format(os.getpid())])
TM = TrainingMonitor(figPath, jsonPath=jsonPath, startAt=0)

RLR = ReduceLROnPlateau(factor=np.sqrt(0.1), cooldown=0, patience=5, min_lr=0.5e-6)

callbacks = [MC, TM, RLR]

4908


In [19]:
history = model.fit(x_train, y_train,
            batch_size=32,
            epochs=100,
            validation_data=(x_val, y_val),
            shuffle="batch",
            callbacks=callbacks)

Train on 700 samples, validate on 150 samples
Epoch 1/100
700/700 [==============================] - 4s 5ms/step - loss: 2.2894 - acc: 0.1940 - val_loss: 2.1784 - val_acc: 0.2000
Epoch 2/100
700/700 [==============================] - 2s 2ms/step - loss: 2.1430 - acc: 0.2369 - val_loss: 2.1156 - val_acc: 0.2520
Epoch 3/100
700/700 [==============================] - 2s 2ms/step - loss: 2.0746 - acc: 0.2694 - val_loss: 2.0149 - val_acc: 0.1993
Epoch 4/100
700/700 [==============================] - 2s 2ms/step - loss: 1.9794 - acc: 0.3111 - val_loss: 1.9261 - val_acc: 0.3033
Epoch 5/100
700/700 [==============================] - 2s 2ms/step - loss: 1.8889 - acc: 0.3697 - val_loss: 1.8381 - val_acc: 0.4153
Epoch 6/100
700/700 [==============================] - 2s 2ms/step - loss: 1.8114 - acc: 0.3873 - val_loss: 1.7689 - val_acc: 0.3813
Epoch 7/100
700/700 [==============================] - 2s 2ms/step - loss: 1.7414 - acc: 0.3944 - val_loss: 1.7019 - val_acc: 0.3673
Epoch 8/100
700/700 [==

In [117]:
scores = model.evaluate(x_test, y_test, verbose=0)
print("Train: %.2f%%; Val: %.2f%%; Test: %.2f%%" % 
      (np.max(history.history['acc'])*100, np.max(history.history['val_acc'])*100, scores[1]*100)
     )

Train: 82.71%; Val: 80.27%; Test: 80.20%


In [118]:
model_json = model.to_json()
with open("../output/saved/date_model_seq2seq_attention_80.20.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights("../output/saved/date_weight_seq2seq_attention_80.20.hdf5")